# Data collection

In this cell, price/volume data will be fetched from the yfinance library. For the purposes of this project, we will focus on SPY due to its popularity as a holding, ease of access to coverage, and its importance as an indicator for US equity market performance.

After the SPY data is fetched, we will use the pandas datareader to import economic data from FRED, and then combine all of the features into a csv file. 

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

# Select SPY as the stock
stock = 'SPY'
data = yf.Ticker(stock)

# Fetch historical data
df = data.history(period="10y", interval="1mo")
df.reset_index(inplace=True)

In [2]:
# Drop less relevant columns
df = df.drop(columns=['Dividends', 'Stock Splits', 'Capital Gains'])
df.head()

,Date,Open,High,Low,Close,Volume
0,2015-02-01 00:00:00-05:00,168.072827,178.314307,166.232887,176.986862,1901638100
1,2015-03-01 00:00:00-05:00,177.087705,178.163102,171.492275,173.433029,2749923000
2,2015-04-01 00:00:00-04:00,174.173445,179.312821,172.586901,175.920334,2036685200
3,2015-05-01 00:00:00-04:00,176.713593,180.409899,174.485686,178.181992,1892519900
4,2015-06-01 00:00:00-04:00,178.857118,180.038579,173.236711,173.717743,2535242600


In [3]:
# Import libraries
from datetime import date
import pandas_datareader.data as web
from dateutil.relativedelta import relativedelta

# Get date range
end_date = date.today()
start_date = end_date - relativedelta(years=10)

# Fetching economic data
cpi = web.DataReader('CPIAUCNS', 'fred', start_date, end_date)  # CPI for All Urban Consumers
unemployment = web.DataReader('UNRATE', 'fred', start_date, end_date)  # Unemployment rate
consumer_conf = web.DataReader('UMCSENT', 'fred', start_date, end_date)  # Consumer Confidence Index
interest_rate = web.DataReader('FEDFUNDS', 'fred', start_date, end_date)  # Federal Funds Effective Rate

# Concatenate all DataFrames on the index (Date)
economic_data = pd.concat([cpi, unemployment, consumer_conf, interest_rate], axis=1)

# Rename columns for clarity
economic_data.columns = ['cpi', 'unemployment', 'consumer_confidence', 'interest_rate']

# Check for missing data and align the DataFrames on Date index
economic_data = economic_data.dropna()  # Drop rows with missing data

# Get Date as a column
economic_data = economic_data.reset_index()
economic_data.rename(columns={'DATE':'Date'}, inplace=True)

print(economic_data.head())

        Date      cpi  unemployment  consumer_confidence  interest_rate
0 2015-02-01  234.722           5.5                 95.4           0.11
1 2015-03-01  236.119           5.4                 93.0           0.11
2 2015-04-01  236.599           5.4                 95.9           0.12
3 2015-05-01  237.805           5.6                 90.7           0.12
4 2015-06-01  238.638           5.3                 96.1           0.13


In [4]:
# Now merge the two DataFrames based on their indices

# Convert to timezone-naive by removing timezone information
df['Date'] = df['Date'].dt.tz_localize(None)
merged_df = pd.concat([df, economic_data], axis=1, join='inner')
merged_df.head()
merged_df.to_csv("data/merged_df.csv")